Objective : Dumps the data into data_dumps database with given table name
    
How can it be useful : We can run queries on a interemediate CTE

How to use:
    
    1. Give the csv_file_path
    2. Give the columns of dates in dates_col_list
    3. Give the table name
    3. Run all cells

## Inputs

In [1]:
csv_file_path = "data/TAT_table_opps_cases_tasksCTE_debugged.csv"
table_name = 'TAT_table_opps_cases_tasks_CTE_debugged'
dates_col_list  = ["case_created_time",'opp_createddate'
                                                          ,'first_connected_time_after_interest'
                                                          ,'first_attempt_time_after_interest']

## Imports

In [2]:
import psycopg2 as pg2
import pandas as pd
import numpy as np
import os
import sys

In [3]:
db_credentials = {"host":"localhost",
               "database":"data_dumps",
               'user':"postgres",
               "password":"123456"}

In [4]:
df = pd.read_csv(csv_file_path,
                 parse_dates = dates_col_list)

In [5]:
col_type_mapper = {"int64": "INT","datetime64[ns]": "date", "object" : "CHAR(100)", 'float64' : 'float'  }
def create_table_query_from_csv(df,table_name):
    
    global col_type_mapper
    
    header_dtype = df.dtypes.to_frame()
    
    create_query_inner = ""
    for i in range(len(header_dtype)):
        col_type = str(header_dtype.iloc[i][0])
        col_name = header_dtype.index[i]
        create_query_inner = create_query_inner + f"{col_name} {col_type_mapper[col_type]}" 
        if i!= len(header_dtype) -1 :
            create_query_inner = create_query_inner + ", \n"


    create_table_query = f"""

    CREATE TABLE {table_name} ( {create_query_inner})

    """
    return create_table_query
    

In [6]:
def create_table_from_csv(csv_file_path,dates_col_list,table_name):
    
    df = pd.read_csv(csv_file_path, parse_dates = dates_col_list)
    
    global db_credentials
    conn = pg2.connect(**db_credentials)
    cursor = conn.cursor()
    
    create_table_query = create_table_query_from_csv(df,table_name)
    cursor.execute(create_table_query)
    print(f"SUCESSFULLY CREATED TABLE {table_name}")
    
    csv_abs_file_path = os.path.join(os.getcwd(),csv_file_path)

    with open(csv_file_path,  'rb') as csv_reader:
        next(csv_reader)
        cursor.copy_expert(f"""COPY {table_name} FROM STDIN WITH (FORMAT CSV)""", csv_reader)


    print(f"SUCESSFULLY COPIED VALUES INTO {table_name}")

    conn.commit()
    conn.close()


In [7]:
create_table_from_csv(csv_file_path,dates_col_list,table_name)

SUCESSFULLY CREATED TABLE TAT_table_opps_cases_tasks_CTE_debugged
SUCESSFULLY COPIED VALUES INTO TAT_table_opps_cases_tasks_CTE_debugged
